# TP5 & 6 - Artifical Intelligence

#### Marlon KUQI & Théo MARIE

### 1 - Understanding convolutional networks for computer vision

In [2]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

# "filters" -> the dimensionality of the output space (i.e. the number of output filters in the convolution).
# "kernel_size" -> An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. 
#                  Can be a single integer to specify the same value for all spatial dimensions.
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(64, (3, 3), activation='relu'))

# Down sampling with MaxPooling2D
# "pool_size" -> integer or tuple of 2 integers, factors by which to downscale (vertical, horizontal). 
#                (2, 2) will halve the input in both spatial dimension. 
#                If only one integer is specified, the same window length will be used for both dimensions.
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 110s 2ms/step - loss: 0.2646 - acc: 0.9185 - val_loss: 0.0549 - val_acc: 0.9816
Epoch 2/12
60000/60000 [==============================] - 111s 2ms/step - loss: 0.0898 - acc: 0.9739 - val_loss: 0.0369 - val_acc: 0.9876
Epoch 3/12
60000/60000 [==============================] - 105s 2ms/step - loss: 0.0661 - acc: 0.9804 - val_loss: 0.0369 - val_acc: 0.9874
Epoch 4/12
60000/60000 [==============================] - 106s 2ms/step - loss: 0.0555 - acc: 0.9831 - val_loss: 0.0311 - val_acc: 0.9890
Epoch 5/12
60000/60000 [==============================] - 108s 2ms/step - loss: 0.0479 - acc: 0.9859 - val_loss: 0.0305 - val_acc: 0.9894
Epoch 6/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0422 - acc: 0.9874 - val_loss: 0.0343 - val_acc: 0.9881
Epoch 7/12
60000/60000 [=========================

In [3]:
# Network structure
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

### 2 - Training a convnet on a small dataset

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

'''
    Training set :
        Dogs : dogs.0 to dogs.599
        Cats : cats.0 to cats.599
        
    Test set : 
        Dogs : dogs.600 to dogs.999
        Cats : cats.600 to cats.999
        
    600 pictures of each category to train our model on
    400 pictures of each category for our model to be tested
    
    Ratio : 66%
'''

img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/test'

nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 1200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 56s 451ms/step - loss: 0.6974 - acc: 0.5520 - val_loss: 0.6526 - val_acc: 0.6238
Epoch 2/50
125/125 [==============================] - 56s 448ms/step - loss: 0.6366 - acc: 0.6235 - val_loss: 0.6865 - val_acc: 0.5875
Epoch 3/50
125/125 [==============================] - 57s 453ms/step - loss: 0.6179 - acc: 0.6930 - val_loss: 0.5811 - val_acc: 0.7050
Epoch 4/50
125/125 [==============================] - 59s 473ms/step - loss: 0.5888 - acc: 0.7005 - val_loss: 0.5817 - val_acc: 0.6950
Epoch 5/50
125/125 [==============================] - 57s 456ms/step - loss: 0.5466 - acc: 0.7325 - val_loss: 0.5441 - val_acc: 0.7362
Epoch 6/50
125/125 [==============================] - 57s 458ms/step - loss: 0.5225 - acc: 0.7500 - val_loss: 0.5236 - val_acc: 0.7612
Epoch 7/50
125/125 [==============================] - 63s 500ms/step - loss: 0.4985 - acc: 0.7600 - val_los